# Textrank

In [41]:
import sqlite3
import pandas as pd

DB_PATH = "enron.db"

experiment = True

conn = sqlite3.connect(DB_PATH)
c = conn.cursor()

if experiment:
    sql_query = "SELECT * FROM N10K" 
else:
    sql_query = "SELECT * FROM N100K" 

df = pd.read_sql_query(sql_query, conn)

conn.commit()
conn.close()

df.columns

Index(['mid', 'sender', 'eid', 'date', 'message_id', 'subject', 'body',
       'folder', 'length_character', 'length_word', 'body_clean',
       'clean_length_character', 'clean_length_word', 'similarities'],
      dtype='object')

In [45]:
from summa import summarizer
from summa import keywords

def get_keywords_safe(text, num_words=8):
    """
    Wrapper for summa.keywords to handle IndexError on short texts.
    Summa throws an IndexError if the text has fewer tokens than 'words'.
    """
    if not isinstance(text, str) or not text.strip():
        return []
    try:
        # Summa returns a newline-separated string
        kw_str = keywords.keywords(text, words=num_words)
        return kw_str.split('\n') if kw_str else []
    except IndexError:
        kw_str = keywords.keywords(text)
        return kw_str.split('\n') if kw_str else []
    except Exception:
        return text.split()

def calculate_query_and_ed(row):
    
    # try: 
        text = f"{row['subject']} \n {row['body_clean']}"

        row['text_rank_query'] = summarizer.summarize(text, words=15) 

        # --- 2. Generate "Elaborative Description" (Keyword Extraction) ---
        # We want a list of salient terms that describe the document content.
        # This creates a "bag of entities" representation useful for GR.
        # row['elaborative_description'] = keywords.keywords(text, words=8)
        row['elaborative_description'] = get_keywords_safe(text, 8)
        
        return row
    # except Exception:
    #     print(row)
    #     raise Exception



In [18]:
row = calculate_query_and_ed(df.iloc[5])
print('='*10)
print('QUERY')
print(row['text_rank_query'])
print('='*10)
print("BODY")
print(row['subject'] + row['body_clean'])
print('='*10)
print('ED')
print(row['elaborative_description'])

QUERY
Re: letter agreement between EES and Enron Corp.
BODY
Re: letter agreement between EES and Enron Corp.---------------------- Forwarded by Sara Shackleton/HOU/ECT on 03/31/2000 
04:08 PM ---------------------------

Sara and Gareth,
We still need to finish this agreement and have it signed.  To my knowledge, 
it has not been completed and signed.
David
Enron Energy Services

Sara,
I'm on someone else's computer - I'm having all sorts of e-mail problems 
today.
The business person on the  EES side to be signing this letter would be Mark 
S. Muller.  He is a Managing Director of EES LLC.
I don't necessarily have a problem with the description of the back-to-back 
obligations, but I'd like to know Gareth's opinion as to whether the language 
is too broad.
David
Sara Shackleton@ECT
03/30/2000 04:38 PM

Attached is a first stab at the agreement.  Please comment.
ED
['letter agreement', 'business person', 'ees', 'llc', 'managing', 'director']


/tmp/ipykernel_60944/2604911314.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row['text_rank_query'] = summarizer.summarize(text, words=15)
/tmp/ipykernel_60944/2604911314.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row['text_rank_query'] = summarizer.summarize(text, words=15)
/tmp/ipykernel_60944/2604911314.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row['elaborative_description'] = keywords.keywords(text, wor

In [21]:
print(df)

         mid                     sender     eid        date  \
0     332279     daren.farmer@enron.com  332114  24/02/2000   
1     466418      carol.clair@enron.com  466022  22/07/2000   
2     153571    chris.germany@enron.com  151808  08/05/2000   
3     489653  kimberly.watson@enron.com  488916  14/08/2001   
4     281696     leann.walton@enron.com  278851  09/11/2001   
...      ...                        ...     ...         ...   
9995  449452    bjspringer@jonesday.com  449405  08/11/2000   
9996  349257       kate.symes@enron.com  348176  30/03/2001   
9997  430195  courtnie.parker@enron.com  430050  14/05/2001   
9998  280639    jeff.dasovich@enron.com  278851  22/08/2001   
9999  100291    jeff.dasovich@enron.com   98475  16/02/2001   

                                         message_id  \
0     <13856275.1075854158988.JavaMail.evans@thyme>   
1     <13282917.1075842132788.JavaMail.evans@thyme>   
2        <80709.1075853802377.JavaMail.evans@thyme>   
3      <4732369.1075855

In [46]:
df_queries = df.apply(lambda x: calculate_query_and_ed(x), axis=1)

KeyboardInterrupt: 

In [42]:
df['char_count'] = df['body_clean'].str.len()

# Sort from short to long
df_sorted = df.sort_values(by='char_count', ascending=True)

# Display result
print(df_sorted.iloc[3]['body_clean'])

Dan just dumped me, so we can go any where you want.  Do you have a 
preference?
Kay

see you then


todo: queries
and queries transformer

In [ ]:

conn = sqlite3.connect(DB_PATH)

# Write to new table 'similarities'
# if_exists='replace' drops the table if it exists and creates a new one
# if_exists='append' adds to it
df.to_sql(
    name='N10k_text_rank',
    con=conn,
    if_exists='replace', 
    index=False,
    chunksize=10000  # Write in batches to save memory
)

cursor = conn.cursor()
cursor.execute("CREATE INDEX IF NOT EXISTS idx_sim_mid ON Message (mid)")
conn.commit()

conn.close()